# Create mock kinematics data for axisymmetric ETG sample

Xiangyu Huang


In [1]:
import numpy as np

import pandas as pd
import corner
from copy import deepcopy
import h5py


In [2]:
from deproject.Profiles.SIS_truncated_physical import SIS_truncated_physical
from deproject.Util.orientation import Inclination, Sphere_random_point
from deproject.Profiles.Jaffe import Jaffe
from deproject.Cosmo.default_cosmo import get_default_lens_cosmo
from deproject.MGE_analysis.mge_proj import MGE_Proj
from deproject.MGE_analysis.intr_mge import Intr_MGE
from deproject.MGE_analysis.mge_misc import sum_gaussian_components

from jampy.jam_axi_proj import jam_axi_proj
from jampy.mge_half_light_isophote import mge_half_light_isophote

from plotbin.plot_velfield import plot_velfield

from mgefit.mge_fit_1d import mge_fit_1d

from lenstronomy.Analysis.lens_profile import LensProfileAnalysis

## Calculation

We use the truncated SIS profile as the mass density profile and Jaffe profile as the stellar tracer.

In [3]:
def get_truncsis_intr_mge(sigma_v, rc_kpc, r_sym, qintr, plot_mge=0, fignum=1):
    """get the amplitude and dispersion of the MGE describing the INTRINSIC mass density/stellar light profile along the symmetry axis 

    Args:
        sigma_v (_type_): sigma_sph of the truncated SIS profile
        rc_kpc (_type_): truncation radius in kpc
        r_sym (_type_): coordinate along the symmetry axis
        qintr (_type_): intrinsic axis ratio. If oblate, qintr < 1; if prolate, qintr > 1
        plot_mge (int, optional): _description_. Defaults to 0.
        fignum (int, optional): _description_. Defaults to 1.

    Returns:
        _type_: amplitude in M_sun/pc^3
        _type_: dispersion in pc
    """
    sis_profile = SIS_truncated_physical(sigma_v=sigma_v, rc = rc_kpc)
    intr_mge = Intr_MGE(profile=sis_profile, qintr=qintr, r_sym=r_sym)
    peak, sigma = intr_mge.MGE_param(kwargs_mge={'ngauss': 20, 'inner_slope': 3, 'outer_slope':1}, plot_mge=plot_mge, fignum=fignum)

    peak = peak / 1e9 # convert to [M_sun/pc^3]
    sigma = sigma * 1e3 # convert to [pc]

    return peak, sigma

In [4]:
def get_proj_mge(mge_proj, distance, inc):
    """get projected MGE

    Args:
        mge_proj (_type_): MGE_proj instance
        distance (_type_): angular diameter distance in Mpc, used to convert the dispersion to arcsec
        inc (_type_): inclination angle [rad]

    Returns:
        _type_: peak of the projected MGE [M_sun/pc^3]
        _type_: sigma of the projected MGE [arcsec]
        _type_: projected axis ratio 
    """
    surf = mge_proj.surf(inc=inc)
    qobs = mge_proj.qobs(inc=inc)

    qobs = np.full_like(surf, qobs)

    pc = distance * np.pi / 0.648 # convert to arcsec
    sigma_intr = mge_proj.sigma
    sigma = sigma_intr / pc

    return surf, sigma, qobs

In [5]:
def get_jaffe_intr_mge(m_star, rs_kpc, qintr, plot_mge=0, fignum=1):

    jaffe_profile = Jaffe(rs=rs_kpc, rho0=1e5)

    r_sym = np.geomspace(0.001, 10 * rs_kpc, 300)

    intr_mge = Intr_MGE(profile=jaffe_profile, qintr=qintr, r_sym=r_sym)

    peak, sigma = intr_mge.MGE_param(kwargs_mge = {'ngauss': 20}, plot_mge=plot_mge, fignum=fignum)

    peak = peak / 1e9 # convert to [M_sun/pc^3]
    sigma = sigma * 1e3 # convert to [pc]

    mtot = intr_mge.MGE_mass_sph()
    peak = m_star / mtot * peak # rescale to desired input stellar mass

    return peak, sigma

In [6]:
def get_sigma_e(surf_lum, sigma_lum, qobs_lum, jam, xbin, ybin, plot_velmap=0, plot_sample_points=0, fignum=1):
    """calculate velocity dispersion within the half-light radius from a jam model

    Args:
        surf_lum (_type_): peak of surface luminosity MGE 
        sigma_lum (_type_): sigma of surface luminocity MGE
        qobs_lum (_type_): array of the projected axis raio of the surface luminosity MGEs
        jam (_type_): jam model, a jampy.jam_axi_proj instance
        xbin (_type_): x coordinate to sample the velocity dispersion
        ybin (_type_): y coordinate to sample the velocity dispersion
        plot_velmap (int, optional): whether to plot the velocity dispersion map. Defaults to 0.
        plot_sample_points (int, optional): whether to plot the xy coordinates within the half-light radius. Defaults to 0.
        fignum (int, optional): _description_. Defaults to 1.

    Raises:
        ValueError: _description_

    Returns:
        _type_: _description_
    """
    ifu_dim = int(np.sqrt(len(xbin)))
    if np.all(qobs_lum <= 1):
        flux = jam.flux
        reff, reff_maj, eps_e, lum_tot = mge_half_light_isophote(surf_lum, sigma_lum, qobs_lum)
    elif np.all(qobs_lum > 1):
        flux = np.reshape(jam.flux, (ifu_dim, ifu_dim)).T  # for prolate rotate the flux map by 90 degrees to calculate the half-light radius
        flux = flux.flatten() 
        reff, reff_maj, eps_e, lum_tot = mge_half_light_isophote(surf_lum, sigma_lum, 1/qobs_lum)
    else:
        raise ValueError('Apparent axis ratio must be constant with radius!')

    w = xbin**2 + (ybin/(1 - eps_e))**2 < reff_maj**2

    model = jam.model

    sig_e = np.sqrt((flux[w]*model[w]**2).sum()/flux[w].sum())

    if plot_velmap:
        plt.figure(fignum)
        plot_velfield(xbin, ybin, model, flux=flux, nodots=1, colorbar=1)
        ax = plt.gca()
        if plot_sample_points:
            ax.plot(xbin[w], ybin[w], ',')
        ax.set_xlabel('arcsec')
        ax.set_ylabel('arcsec')
    return sig_e

In [7]:
def get_sigma_e_dist(catalog, oblate, single_proj, save_mge=0):
    """calculate velocity dispersion distribution under many projections

    Args:
        catalog (_type_): _description_
        oblate (_type_): True if intrinsic shape is oblate; False if intrinsic shape is prolate
        single_proj (_type_): number of projections for a single halo
        save_mge (int, optional): whether to return the MGE components. Defaults to 0.

    Returns:
        _type_: _description_
    """
    # set up a cosmology and lens_source configuration
    lens_cosmo = get_default_lens_cosmo()
    distance = lens_cosmo.Dd
    sigma_crit = lens_cosmo.Sigma_crit / 1e12 # [M_sun/pc^2]

    num_galaxy = len(catalog)
    num_proj = num_galaxy * single_proj
    sigma_e_all = np.zeros(num_proj)
    qobs_all = np.zeros(num_proj)
    theta_e_all = np.zeros(num_proj)

    if save_mge:
        peak_mge_den_all = np.zeros(shape=(num_proj, 20))
        sigma_mge_den_all = np.zeros_like(peak_mge_den_all)
        peak_mge_lum_all = np.zeros(shape=(num_proj, 20))
        sigma_mge_lum_all = np.zeros_like(peak_mge_lum_all)

    # access catalog info
    sigma_rm_all = catalog['sigma_random_los'].values
    Re_kpc_all = catalog['Re'].values
    if oblate:
        qintr_all = catalog['xi_new'].values
    else:
        qintr_all = 1/catalog['zeta_new'].values
    # compute rc_kpc for truncated sis profile
    theta_e_sis_all = lens_cosmo.sis_sigma_v2theta_E(sigma_rm_all)
    theta_e_sis_kpc_all = lens_cosmo.arcsec2Mpc_lens(theta_e_sis_all) * 1e3
    rc_kpc_all = theta_e_sis_kpc_all * 200

    # compute effective radius in arcsec
    Re_arcsec_all = lens_cosmo.Mpc2arcsec_lens(Re_kpc_all / 1e3)

    # sample inclination angle uniformly on a sphere
    theta_all, phi_all = Sphere_random_point(num_proj)
    inc_all = Inclination(oblate=oblate, theta=theta_all, phi=phi_all, deg=0)
    inc_deg_all = Inclination(oblate=oblate, theta=theta_all, phi=phi_all, deg=1)

    for i in range(num_galaxy):

        sigma_rm = sigma_rm_all[i]
        rs_kpc = Re_kpc_all[i] / 0.763 
        qintr = qintr_all[i]
        rc_kpc = rc_kpc_all[i]
        theta_e_sis = theta_e_sis_all[i]
        Re_arcsec = Re_arcsec_all[i]

        # intrinsic MGE component of density and luminosity
        r_sym = np.geomspace(0.01, 5 * rc_kpc, 300)
        r_fine = np.geomspace(0.01, 10 * theta_e_sis, 100)
        peak_den, sigma_den = get_truncsis_intr_mge(sigma_rm, rc_kpc, r_sym, qintr)
        peak_lum, sigma_lum = get_jaffe_intr_mge(1e11, rs_kpc, qintr)

        beta = np.zeros_like(peak_lum)

        # project each single_proj times
        mge_proj_den = MGE_Proj(peak_den, sigma_den, qintr)
        mge_proj_lum = MGE_Proj(peak_lum, sigma_lum, qintr)

        # make a grid
        xx = np.linspace(-4 * theta_e_sis, 4 * theta_e_sis, 100)  # avoid (x,y)=(0,0)
        xbin, ybin = map(np.ravel, np.meshgrid(xx, xx))

        for j in range(single_proj):
            
            inc = inc_all[single_proj*i + j]
            inc_deg = inc_deg_all[single_proj*i + j]
            peak_surf_den, sigma_surf_den, qobs_den = get_proj_mge(mge_proj_den, distance, inc)
            peak_surf_lum, sigma_surf_lum, qobs_lum = get_proj_mge(mge_proj_lum, distance, inc)

            # save the mge
            if save_mge:
                peak_mge_den_all[single_proj*i + j, :len(peak_surf_den)] =  peak_surf_den
                sigma_mge_den_all[single_proj*i + j, :len(sigma_surf_den)] = sigma_surf_den
                peak_mge_lum_all[single_proj*i + j, :len(peak_surf_lum)] = peak_surf_lum
                sigma_mge_lum_all[single_proj*i + j, :len(sigma_surf_lum)] = sigma_surf_lum

            # compute observed axis ratio 
            Qobs = mge_proj_den.qobs(inc=inc)
            qobs_all[single_proj*i + j] = Qobs

            # compute einstein radius from the radial profile
            surf_den_circ = sum_gaussian_components(r_fine/np.sqrt(Qobs), peak_surf_den, sigma_surf_den) # circularized surface density profile
            theta_e_jam = LensProfileAnalysis.effective_einstein_radius_from_radial_profile(r_fine, surf_den_circ/sigma_crit)
            theta_e_all[single_proj*i + j] = theta_e_jam
            print(f"theta_E: {theta_e_jam:.3f} arcsec")

            # jam prediction of LoS velocity dispersion
            jam = jam_axi_proj(peak_surf_lum, sigma_surf_lum, qobs_lum, peak_surf_den, sigma_surf_den, qobs_den, inc_deg, 0, distance, xbin, ybin, plot=0, beta=beta, align='sph')
            sigma_e = get_sigma_e(peak_surf_lum, sigma_surf_lum, qobs_lum, jam, xbin, ybin)
            print(f"sigma_e: {sigma_e:.2f} km/s")
            sigma_e_all[single_proj*i + j] = sigma_e

    data = np.vstack([theta_e_all, qobs_all, sigma_e_all, inc_deg_all, theta_all, phi_all])

    if save_mge:
        return data, peak_mge_den_all, sigma_mge_den_all, peak_mge_lum_all, sigma_mge_lum_all
    else:
        return data

## Access data

Data is axisymmetric shape catalog, which is modified from the original triaxial TNG ETG catalog. For the oblate sample with $T < 0.5$, we take $b/a \rightarrow 1$, $c/a \rightarrow (1 + b/a) * c/a / 2$ (preserving total volumn of the ellipsoid). For the prolate sample with $T > 0.5$, we take $b/a = c/a \rightarrow (b/a + c/a) / 2$.

In [8]:
cat_oblate = pd.read_pickle('./axisym_tng_catalog/oblate_catalog.pkl')
cat_prolate = pd.read_pickle('./axisym_tng_catalog/prolate_catalog.pkl')

## Mock data for oblate TNG100 ETG

In [9]:
rerun_oblate = False

if rerun_oblate:

    data_oblate, peak_mge_den, sigma_mge_den, peak_mge_lum, sigma_mge_lum = get_sigma_e_dist(cat_oblate, 1, 10, save_mge=1)
    np.save('./tng_kin_oblate.npy', data_oblate)

    with h5py.File('./tng_kin_mge_oblate.hdf5', 'w') as f:
        f.create_dataset(name='peak_den', data = peak_mge_den)
        f.create_dataset(name='sigma_den', data = sigma_mge_den)
        f.create_dataset(name='peak_lum', data = peak_mge_lum)
        f.create_dataset(name='sigma_lum', data = sigma_mge_lum)

## Mock data for prolate TNG100 ETG

In [10]:
rerun_prolate = False

if rerun_prolate:

    data_prolate, peak_mge_den_prolate, sigma_mge_den_prolate, peak_mge_lum_prolate, sigma_mge_lum_prolate = get_sigma_e_dist(cat_prolate, 0, 10, save_mge=1)
    np.save('./tng_kin_prolate.npy', data_prolate)

    with h5py.File('./tng_kin_mge_prolate.hdf5', 'w') as f:
        f.create_dataset(name='peak_den', data = peak_mge_den_prolate)
        f.create_dataset(name='sigma_den', data = sigma_mge_den_prolate)
        f.create_dataset(name='peak_lum', data = peak_mge_lum_prolate)
        f.create_dataset(name='sigma_lum', data = sigma_mge_lum_prolate)